Day 16 (https://adventofcode.com/2023/day/16)

In [1]:
import numpy as np
from copy import deepcopy

with open('./inputs/day16.txt', 'r') as f:
    tile_map = np.array([list(line.strip()) for line in f.readlines()])

    beam_history_pt1 = []
    beams = [(0, 0, 'E')]

    while len(beams) > 0:
        beam = beams.pop()
        if beam in beam_history_pt1:
            continue

        i, j = beam[0], beam[1]
        beam_history_pt1.append(beam)
        if (beam[2] == 'E' and tile_map[i, j] in ('/', '|')) or \
            (beam[2] == 'W' and tile_map[i, j] in ('\\', '|')) or \
            (beam[2] == 'N' and tile_map[i, j] in ('.', '|')):
            if i > 0:
                beams.append((i-1, j, 'N'))
        if (beam[2] == 'E' and tile_map[i, j] in ('\\', '|')) or \
            (beam[2] == 'W' and tile_map[i, j] in ('/', '|')) or \
            (beam[2] == 'S' and tile_map[i, j] in ('.', '|')):
            if i+1 < tile_map.shape[0]:
                beams.append((i+1, j, 'S'))
        if (beam[2] == 'S' and tile_map[i, j] in ('/', '-')) or \
            (beam[2] == 'N' and tile_map[i, j] in ('\\', '-')) or \
            (beam[2] == 'W' and tile_map[i, j] in ('.', '-')):
            if j > 0:
                beams.append((i, j-1, 'W'))
        if (beam[2] == 'S' and tile_map[i, j] in ('\\', '-')) or \
            (beam[2] == 'N' and tile_map[i, j] in ('/', '-')) or \
            (beam[2] == 'E' and tile_map[i, j] in ('.', '-')):
            if j+1 < tile_map.shape[1]:
                beams.append((i, j+1, 'E'))

    print('Answer to Day 16, Part 1:', len(set(((b[0], b[1]) for b in beam_history_pt1))))

    def get_beam_path_til_split(init_beam):
        beam_path = [init_beam]
        while True:
            beam = beam_path[-1]
            i, j = beam[0], beam[1]
            if (beam[2] == 'E' and tile_map[i, j] == '/') or \
                (beam[2] == 'W' and tile_map[i, j] == '\\') or \
                (beam[2] == 'N' and tile_map[i, j] in ('.', '|')):
                if i > 0:
                    beam_path.append((i-1, j, 'N'))
                    continue
            elif (beam[2] == 'E' and tile_map[i, j] == '\\') or \
                (beam[2] == 'W' and tile_map[i, j] == '/') or \
                (beam[2] == 'S' and tile_map[i, j] in ('.', '|')):
                if i+1 < tile_map.shape[0]:
                    beam_path.append((i+1, j, 'S'))
                    continue
            elif (beam[2] == 'S' and tile_map[i, j] == '/') or \
                (beam[2] == 'N' and tile_map[i, j] == '\\') or \
                (beam[2] == 'W' and tile_map[i, j] in ('.', '-')):
                if j > 0:
                    beam_path.append((i, j-1, 'W'))
                    continue
            elif (beam[2] == 'S' and tile_map[i, j] == '\\') or \
                (beam[2] == 'N' and tile_map[i, j] == '/') or \
                (beam[2] == 'E' and tile_map[i, j] in ('.', '-')):
                if j+1 < tile_map.shape[1]:
                    beam_path.append((i, j+1, 'E'))
                    continue

            return beam_path, (i, j) if (tile_map[i, j]=='-' and beam[2] in ('N', 'S')) or (tile_map[i, j]=='|' and beam[2] in ('E', 'W')) else None
        
    split_dict = {}
    for i, j in np.argwhere(tile_map=='-'):
        east_path, east_split = get_beam_path_til_split((i, j, 'E'))
        west_path, west_split = get_beam_path_til_split((i, j, 'W'))
        split_dict[(i, j)] = {
            'path': east_path + west_path,
            'splits': [s for s in [east_split, west_split] if s is not None]
        }

    for i, j in np.argwhere(tile_map=='|'):
        north_path, north_split = get_beam_path_til_split((i, j, 'N'))
        south_path, south_split = get_beam_path_til_split((i, j, 'S'))
        split_dict[(i, j)] = {
            'path': north_path + south_path,
            'splits': [s for s in [north_split, south_split] if s is not None]
        }

    def get_beam_path(beam):
        path, split = get_beam_path_til_split(beam)
        splits = set([split])
        last_num_splits = 0
        while len(splits) != last_num_splits:
            last_num_splits = len(splits)
            for split in deepcopy(splits):
                if split is None:
                    continue

                splits.update(split_dict[split]['splits'])

        for s in splits:
            if split is None:
                continue
            path += split_dict[s]['path']

        return path
    
    init_beams = [(0, j, 'S') for j in range(tile_map.shape[1])]
    init_beams += [(tile_map.shape[0]-1, j, 'N') for j in range(tile_map.shape[1])]
    init_beams += [(i, 0, 'E') for i in range(tile_map.shape[0])]
    init_beams += [(i, tile_map.shape[1]-1, 'W') for i in range(tile_map.shape[0])]

    num_tiles = [len(set((b[0], b[1]) for b in get_beam_path(init_beam))) for init_beam in init_beams]
    print('Answer to Day 16, Part 2:', max(num_tiles))

Answer to Day 16, Part 1: 8539
Answer to Day 16, Part 2: 8674


Day 17 (https://adventofcode.com/2023/day/17)

In [2]:
import numpy as np
import networkx as nx

with open('./inputs/day17.txt', 'r') as f:
    tiles = np.array([list(map(int, l.strip())) for l in f.readlines()])

    # part 1
    graph = nx.DiGraph()
    for i in range(tiles.shape[0]):
        for j in range(tiles.shape[1]):
            # node entered from north or south
            graph.add_node((i, j, 'NS'), heat_loss=tiles[i, j])
            # node entered from east or west
            graph.add_node((i, j, 'EW'), heat_loss=tiles[i, j])
            for k in (1, 2, 3):
                if i-k >= 0:
                    graph.add_edge((i, j, 'EW'), (i-k, j, 'NS'), weight=np.sum(tiles[i-k:i, j]))
                    graph.add_edge((i-k, j, 'EW'), (i, j, 'NS'), weight=np.sum(tiles[i-k+1:i+1, j]))
                if j-k >= 0:
                    graph.add_edge((i, j, 'NS'), (i, j-k, 'EW'), weight=np.sum(tiles[i, j-k:j]))
                    graph.add_edge((i, j-k, 'NS'), (i, j, 'EW'), weight=np.sum(tiles[i, j-k+1:j+1]))

    graph.add_node('start')
    graph.add_edge('start', (0, 0, 'NS'), weight=0)
    graph.add_edge('start', (0, 0, 'EW'), weight=0)

    graph.add_node('end')
    graph.add_edge((tiles.shape[0]-1, tiles.shape[1]-1, 'NS'), 'end', weight=0)
    graph.add_edge((tiles.shape[0]-1, tiles.shape[1]-1, 'EW'), 'end', weight=0)

    p = nx.bidirectional_dijkstra(
        graph,
        source='start',
        target='end',
        weight=lambda from_node, to_node, edge: edge['weight']
    )
    print('Answer to Day 17, Part 1:', p[0])

    # part 2: not too much of a change!
    graph = nx.DiGraph()
    for i in range(tiles.shape[0]):
        for j in range(tiles.shape[1]):
            # node entered from north or south
            graph.add_node((i, j, 'NS'), heat_loss=tiles[i, j])
            # node entered from east or west
            graph.add_node((i, j, 'EW'), heat_loss=tiles[i, j])
            for k in (4, 5, 6, 7, 8, 9, 10):
                if i-k >= 0:
                    graph.add_edge((i, j, 'EW'), (i-k, j, 'NS'), weight=np.sum(tiles[i-k:i, j]))
                    graph.add_edge((i-k, j, 'EW'), (i, j, 'NS'), weight=np.sum(tiles[i-k+1:i+1, j]))
                if j-k >= 0:
                    graph.add_edge((i, j, 'NS'), (i, j-k, 'EW'), weight=np.sum(tiles[i, j-k:j]))
                    graph.add_edge((i, j-k, 'NS'), (i, j, 'EW'), weight=np.sum(tiles[i, j-k+1:j+1]))

    graph.add_node('start')
    graph.add_edge('start', (0, 0, 'NS'), weight=0)
    graph.add_edge('start', (0, 0, 'EW'), weight=0)

    graph.add_node('end')
    graph.add_edge((tiles.shape[0]-1, tiles.shape[1]-1, 'NS'), 'end', weight=0)
    graph.add_edge((tiles.shape[0]-1, tiles.shape[1]-1, 'EW'), 'end', weight=0)

    p = nx.bidirectional_dijkstra(
        graph,
        source='start',
        target='end',
        weight=lambda from_node, to_node, edge: edge['weight']
    )
    print('Answer to Day 17, Part 2:', p[0])

Answer to Day 17, Part 1: 907
Answer to Day 17, Part 2: 1057


Day 18 (https://adventofcode.com/2023/day/18)

In [3]:
import numpy as np

with open('./inputs/day18.txt', 'r') as f:
    lines = [l.split() for l in f.readlines()]
    path = [(0, 0)]
    for l in lines:
        last_point = path[-1]
        if l[0] == 'R':
            path.append((last_point[0], last_point[1]+int(l[1])))
        elif l[0] == 'L':
            path.append((last_point[0], last_point[1]-int(l[1])))
        elif l[0] == 'U':
            path.append((last_point[0]-int(l[1]), last_point[1]))
        elif l[0] == 'D':
            path.append((last_point[0]+int(l[1]), last_point[1]))

    xmin = min(p[0] for p in path)
    xmax = max(p[0] for p in path)
    ymin = min(p[1] for p in path)
    ymax = max(p[1] for p in path)

    dig_map = np.zeros((xmax-xmin+1, ymax-ymin+1))
    last_point = (-xmin, -ymin)
    for l in lines:
        if l[0] == 'R':
            dig_map[last_point[0], last_point[1]:last_point[1]+int(l[1])+1] = 1
            last_point = (last_point[0], last_point[1]+int(l[1]))
        elif l[0] == 'L':
            dig_map[last_point[0], last_point[1]-int(l[1]):last_point[1]] = 1
            last_point = (last_point[0], last_point[1]-int(l[1]))
        elif l[0] == 'U':
            dig_map[last_point[0]-int(l[1]):last_point[0], last_point[1]] = 1
            last_point = (last_point[0]-int(l[1]), last_point[1])
        elif l[0] == 'D':
            dig_map[last_point[0]:last_point[0]+int(l[1])+1, last_point[1]] = 1
            last_point = (last_point[0]+int(l[1]), last_point[1])

    dig_map = np.pad(dig_map, pad_width=1, constant_values=9)
    stuff = list(map(tuple, np.argwhere(dig_map==9)))
    while len(stuff) > 0:
        i, j = stuff.pop()
        points_to_check = ((max(0, i-1), j), (min(dig_map.shape[0]-1, i+1), j), (i, max(0, j-1)), (i, min(dig_map.shape[1]-1, j+1)))
        for p in points_to_check:
            if dig_map[p] == 0:
                dig_map[p] = 9
                stuff.append(p)

    print('Answer to Day 18, Part 1:', np.sum(dig_map < 8))

    # part 2: problem just got a whole lot bigger
    path = [(0, 0)]
    for l in lines:
        last_point = path[-1]
        if l[2][-2] == '0':
            path.append((last_point[0], last_point[1]+int(l[2][2:-2], 16)))
        elif l[2][-2] == '1':
            path.append((last_point[0]+int(l[2][2:-2], 16), last_point[1]))
        elif l[2][-2] == '2':
            path.append((last_point[0], last_point[1]-int(l[2][2:-2], 16)))
        elif l[2][-2] == '3':
            path.append((last_point[0]-int(l[2][2:-2], 16), last_point[1]))

    xmin = min(p[0] for p in path)
    xmax = max(p[0] for p in path)
    ymin = min(p[1] for p in path)
    ymax = max(p[1] for p in path)

    x_breaks = sorted(list(set(p[0] for p in path)))
    y_breaks = sorted(list(set(p[1] for p in path)))
    x_breaks.append(xmax-xmin+1)
    y_breaks.append(ymax-ymin+1)

    def point_is_on_path(point):
        return any(
            (
                path[i][0] == path[i+1][0] and
                path[i][0] == point[0] and
                point[1] <= max(path[i][1], path[i+1][1]) and
                point[1] >= min(path[i][1], path[i+1][1])
            ) or (
                path[i][1] == path[i+1][1] and
                path[i][1] == point[1] and
                point[0] <= max(path[i][0], path[i+1][0]) and
                point[0] >= min(path[i][0], path[i+1][0])
            )
            for i in range(len(path)-1)
        )

    part2_ans = 0
    # handle *between* breakpoints
    for i in range(len(x_breaks)-1):
        if x_breaks[i+1] - x_breaks[i] <= 1:
            continue

        inside = False
        for j in range(len(y_breaks)-1):
            if point_is_on_path((x_breaks[i]+1, y_breaks[j])):
                inside = not inside
                if not inside:
                    part2_ans += (x_breaks[i+1]-x_breaks[i]-1)
            if inside:
                part2_ans += ((x_breaks[i+1]-x_breaks[i]-1) * (y_breaks[j+1]-y_breaks[j]))

    # handle *on* the breakpoints
    for i in range(len(x_breaks)-1):
        inside = False
        num_points = 0
        last_point_direction = 0
        for j in range(len(y_breaks)-1):
            if point_is_on_path((x_breaks[i], y_breaks[j])):
                if (x_breaks[i], y_breaks[j]) in path:
                    num_points += 1
                    if num_points%2 == 0:
                        if (point_is_on_path((x_breaks[i]+1, y_breaks[j])) and last_point_direction==-1) or \
                            (point_is_on_path((x_breaks[i]-1, y_breaks[j])) and last_point_direction==1):
                            inside = not inside
                            last_point_direction = 0
                        
                        if not inside:
                            part2_ans += 1

                    elif point_is_on_path((x_breaks[i]+1, y_breaks[j])):
                        last_point_direction = 1
                    else:
                        last_point_direction = -1
                elif num_points%2 == 0:
                    if inside:
                        part2_ans += 1
                    inside = not inside

            if inside or num_points%2 == 1:
                part2_ans += y_breaks[j+1] - y_breaks[j]

    print('Answer to Day 18, Part 2:', part2_ans)

Answer to Day 18, Part 1: 62365
Answer to Day 18, Part 2: 159485361249806


Day 19 (https://adventofcode.com/2023/day/19)

In [4]:
with open('./inputs/day19.txt', 'r') as f:
    input_stuff = f.read().split('\n\n')
    workflows = {x.split('{')[0]: x.strip('}').split(r'{')[1].split(',') for x in input_stuff[0].split('\n')}
    items = [{x.split('=')[0]: int(x.split('=')[1]) for x in i.strip(r'{}').split(',')} for i in input_stuff[1].split('\n')]

    def is_accepted(item):
        current_workflow = 'in'
        while current_workflow not in ('R', 'A'):
            for condition in workflows[current_workflow]:
                if ':' in condition:
                    if eval(f'{item[condition[0]]}{condition.split(":")[0][1:]}'):
                        current_workflow = condition.split(":")[1]
                        break
                else:
                    current_workflow = condition
                    break

        return current_workflow == 'A'
    
    print('Answer to Day 19, Part 1:', sum(sum(item.values()) for item in filter(is_accepted, items)))

    def range_intersect(r1, r2):
        output = (max(r1[0], r2[0]), min(r1[1], r2[1]))
        if output[0] <= output[1]:
            return output
        else:
            return None
        
    def split_item_ranges(item_range, condition):
        attribute = condition[0]

        if '>' in condition:
            pass_range = (int(condition.split(':')[0].split('>')[1])+1, 4000)
            fail_range = (1, int(condition.split(':')[0].split('>')[1]))
        elif '<' in condition:
            pass_range = (1, int(condition.split(':')[0].split('<')[1])-1)
            fail_range = (int(condition.split(':')[0].split('<')[1]), 4000)

        pass_range_intersection = range_intersect(item_range[attribute], pass_range)
        fail_range_intersection = range_intersect(item_range[attribute], fail_range)
        
        pass_item_range = item_range.copy()
        pass_item_range[attribute] = pass_range_intersection
        fail_item_range = item_range.copy()
        fail_item_range[attribute] = fail_range_intersection

        return pass_item_range if pass_range_intersection is not None else None, \
            fail_item_range if fail_range_intersection is not None else None

    workflow_queue = {}
    for wf in workflows:
        workflow_queue[wf] = []
    workflow_queue['in'].append({'x': (1, 4000), 'm': (1, 4000), 'a': (1, 4000), 's': (1, 4000)})
    workflow_queue['A'] = []
    workflow_queue['R'] = []

    keep_going = True
    while keep_going:
        keep_going = False
        for workflow in workflow_queue:
            if workflow in ('A', 'R') or len(workflow_queue[workflow])==0:
                continue

            keep_going = True
            item_range = workflow_queue[workflow].pop()
            for condition in workflows[workflow]:
                dest_workflow = condition.split(':')[-1]
                if ':' not in condition:
                    workflow_queue[dest_workflow].append(item_range)
                else:
                    pass_item_range, fail_item_range = split_item_ranges(item_range, condition)
                    if pass_item_range is not None:
                        workflow_queue[dest_workflow].append(pass_item_range)
                    if fail_item_range is not None:
                        item_range = fail_item_range

    part2_ans = 0
    for item_range in workflow_queue['A']:
        a = 1
        for r in item_range.values():
            a *= (r[1] - r[0] + 1)

        part2_ans += a

    print('Answer to Day 19, Part 2:', part2_ans)

Answer to Day 19, Part 1: 432427
Answer to Day 19, Part 2: 143760172569135


Day 20 (https://adventofcode.com/2023/day/20)

In [5]:
import functools

# high pulse = 1, low pulse = 0
num_high_pulses = 0
num_low_pulses = 0
action_queue = []
keep_going = True

class FlipFlop:
    def __init__(self, name):
        self.name = name
        self.is_on = False
        self.listeners = []

    def add_listeners(self, listeners):
        for l in listeners:
            self.listeners.append(l)

    def add_inputers(self, inputers):
        pass

    def handle_signal(self, signal, source):
        global num_low_pulses
        global num_high_pulses
        if signal==1:
            num_high_pulses += 1
        else:
            num_low_pulses += 1
            self.is_on = not self.is_on
            for listener in self.listeners:
                action_queue.append(
                    functools.partial(
                        listener.handle_signal,
                        signal=int(self.is_on),
                        source=self
                    )
                )

class Conjunction:
    def __init__(self, name):
        self.name = name
        self.remembered_pulses = {}
        self.listeners = []
        self.inputers = []

    def add_listeners(self, listeners):
        for l in listeners:
            self.listeners.append(l)

    def add_inputers(self, inputers):
        for i in inputers:
            self.inputers.append(i)
            self.remembered_pulses[i.name] = 0

    def handle_signal(self, signal, source):
        global num_low_pulses
        global num_high_pulses
        self.remembered_pulses[source.name] = signal
        if signal==1:
            num_high_pulses += 1
        else:
            num_low_pulses += 1

        for listener in self.listeners:
            action_queue.append(
                functools.partial(
                    listener.handle_signal,
                    signal=int(not all(v==1 for v in self.remembered_pulses.values())),
                    source=self
                )
            )

class Broadcaster:
    def __init__(self, name):
        self.name = name
        self.listeners = []

    def add_listeners(self, listeners):
        for l in listeners:
            self.listeners.append(l)

    def add_inputers(self, inputers):
        pass

    def handle_signal(self, signal, source):
        global num_low_pulses
        global num_high_pulses
        if signal==1:
            num_high_pulses += 1
        else:
            num_low_pulses += 1

        for listener in self.listeners:
            action_queue.append(
                functools.partial(
                    listener.handle_signal,
                    signal=signal,
                    source=self
                )
            )

class GenericModule:
    def __init__(self, name):
        self.name = name

    def add_listeners(self, listeners):
        pass

    def add_inputers(self, inputers):
        pass

    def handle_signal(self, signal, source):
        global num_low_pulses
        global num_high_pulses
        if signal==1:
            num_high_pulses += 1
        else:
            num_low_pulses += 1


class FinalModule:
    def __init__(self, name):
        self.name = name
        self.inputers = []

    def add_listeners(self, listeners):
        pass

    def add_inputers(self, inputers):
        for i in inputers:
            self.inputers.append(i)

    def handle_signal(self, signal, source):
        global num_low_pulses
        global num_high_pulses
        global keep_going
        if signal==1:
            num_high_pulses += 1
        else:
            num_low_pulses += 1
            keep_going = False


with open('./inputs/day20.txt', 'r') as f:
    lines= f.read().split('\n')
    modules = {}

    # create the modules
    for line in lines:
        name = line.split()[0]
        if name[0] == '%':
            modules[name[1:]] = FlipFlop(name=name[1:])
        elif name[0] == '&':
            modules[name[1:]] = Conjunction(name=name[1:])
        elif name == 'broadcaster':
            modules[name] = Broadcaster(name=name)

    # now connect them all up
    for line in lines:
        name = line.split()[0].split('%')[-1].split('&')[-1]
        listeners = [l.strip() for l in line.split('->')[1].split(',')]
        for listener in listeners:
            if listener not in modules.keys():
                modules[listener] = GenericModule(name=name[1:])

            modules[name].add_listeners([modules[listener]])
            modules[listener].add_inputers([modules[name]])

    num_button_pushes = 1000
    for i in range(num_button_pushes):
        action_queue.append(
            functools.partial(
                modules['broadcaster'].handle_signal,
                signal=0,
                source=GenericModule(name='potato')
            )
        )
        while len(action_queue) > 0:
            action_queue.pop(0)()

    print('Answer to Day 20, Part 1:', num_low_pulses*num_high_pulses)

    # create the modules
    modules = {}
    modules['rx'] = FinalModule('rx')
    for line in lines:
        name = line.split()[0]
        if name[0] == '%':
            modules[name[1:]] = FlipFlop(name=name[1:])
        elif name[0] == '&':
            modules[name[1:]] = Conjunction(name=name[1:])
        elif name == 'broadcaster':
            modules[name] = Broadcaster(name=name[1:])

    # now connect them all up
    for line in lines:
        name = line.split()[0].split('%')[-1].split('&')[-1]
        listeners = [l.strip() for l in line.split('->')[1].split(',')]
        for listener in listeners:
            modules[name].add_listeners([modules[listener]])
            modules[listener].add_inputers([modules[name]])

    # there's definitely some manual checking and vague hand-waviness that went into this part
    # but I got the right answer eventually, so I don't really care
    num_button_pushes = 0
    conj_inputs = {i.name: 0 for i in modules['rx'].inputers[0].inputers}
    while any(v == 0 for v in conj_inputs.values()):
        action_queue.append(
            functools.partial(
                modules['broadcaster'].handle_signal,
                signal=0,
                source=GenericModule(name='potato')
            )
        )
        num_button_pushes += 1
        while len(action_queue) > 0 and keep_going:
            f = action_queue.pop(0)
            if f.keywords['source'].name in conj_inputs.keys() and f.keywords['signal'] == 1:
                conj_inputs[f.keywords['source'].name] = num_button_pushes
            
            f()

    part2_ans = 1
    for v in conj_inputs.values():
        part2_ans *= v
    print('Answer to Day 20, Part 2:', part2_ans)

Answer to Day 20, Part 1: 743871576
Answer to Day 20, Part 2: 244151741342687
